# Imports

In [1]:
import os
import pandas as pd
import time

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from tqdm import tqdm

import ollama

import tiktoken

In [2]:
# Force CUDA usage
os.environ["OLLAMA_BACKEND"] = "cuda"
os.environ["OLLAMA_NUM_THREADS"] = "16"

### Parameters

In [3]:
# List of models
ml_models = [
    ('Logistic Regression', LogisticRegression(random_state=42, max_iter=1000)), # 2 sec
    ('Random Forest', RandomForestClassifier(random_state=42)), # 2 min
    ('SVM', SVC(random_state=42)), # 30 min
    ('KNN', KNeighborsClassifier()), # 30 sec
    ('Gradient Boosting', GradientBoostingClassifier(random_state=42)) # 30 sec
]

In [4]:
# LLMs
models = ['llama3.2:1b', 'llama3.2:3b', 'gemma3:1b', 'gemma3:4b', 'llama3.1', 'dolphin3', 'mistral', 'deepseek-llm']

In [5]:
# Vectorize (Bag of words representation)
vectorizer = CountVectorizer(stop_words='english', max_features=10000, ngram_range=(1, 2))

In [6]:
# Number of rows that LLM will generate and compare to
n_rows = 5000

# Load data

In [7]:
# Load the CSV file into a pandas DataFrame
base_df = pd.read_csv('IMB_preprocessed_2025_04_06.csv')

# We want to be able to read the full reviews
pd.set_option('display.max_colwidth', None) 

# Display the first 5 records
base_df.head()

,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.\nThe first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.\nIt is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.\nI would say the main appeal of the show is due to the fact that it goes where other shows wouldn't dare. Forget pretty pictures painted for mainstream audiences, forget charm, forget romance...OZ doesn't mess around. The first episode I ever saw struck me as so nasty it was surreal, I couldn't say I was ready for it, but as I watched more, I developed a taste for Oz, and got accustomed to the high levels of graphic violence. Not just violence, but injustice (crooked guards who'll be sold out for a nickel, inmates who'll kill on order and get away with it, well mannered, middle class inmates being turned into prison bitches due to their lack of street skills or prison experience) Watching Oz, you may become comfortable with what is uncomfortable viewing....thats if you can get in touch with your darker side.",1
1,"A wonderful little production. \nThe filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. \nThe actors are extremely well chosen- Michael Sheen not only ""has got all the polari"" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master's of comedy and his life. \nThe realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional 'dream' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell's murals decorating every surface) are terribly well done.",1
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.\nThis was the most I'd laughed at one of Woody's comedies in years (dare I say a decade?). While I've never been impressed with Scarlet Johanson, in this she managed to tone down her ""sexy"" image and jumped right into a average, but spirited young woman.\nThis may not be the crown jewel of his career, but it was wittier than ""Devil Wears Prada"" and more interesting than ""Superman"" a great comedy to go see with friends.",1
3,"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.\nThis movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.\nOK, first of all when you're going to make a film you must Decide if its a thriller or a drama! As a dram

# Meassure the effectivity of compression
This will be the baseline number of tokens

In [8]:
# Load an encoding (model)
def load_encoding(model_name):
    encoding = tiktoken.encoding_for_model(model_name)
    return encoding

# Count tokens from a string
def count_tokens_from_string(text, encoding_name="gpt-3.5-turbo"):
    encoding = load_encoding(encoding_name)
    tokens = encoding.encode(text)
    num_tokens = len(tokens)
    return num_tokens

In [9]:
# For the complete df
baseline_total_tokens= base_df['review'].apply(count_tokens_from_string).sum()
print("Total tokens in df['review']:", baseline_total_tokens) # 14.2 milion

# For the first n_rows rows
baseline_1k_rows_tokens= base_df['review'][:n_rows].apply(count_tokens_from_string).sum()
print("Total tokens in df['review']:", baseline_1k_rows_tokens) # 28.6 thousand

Total tokens in df['review']: 14274751
Total tokens in df['review']: 1426591


# Experimential setup

In [10]:
def experimental_setup(df):
    # Split into X and y
    X = df['review']
    y = df['sentiment']
    
    # Split data into train and test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Only train on the first n_rows
    X_train = X_train[:n_rows]
    y_train = y_train[:n_rows]

    # Transform text reviews to Bag of Words representation
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)
    
    # Print the shapes of the sets
    print(f"Training Set: X_train shape = {X_train.shape}, y_train shape = {y_train.shape}")
    print(f"Test Set: X_test shape = {X_test.shape}, y_test shape = {y_test.shape}")
    print()

    return X_train, y_train, X_test, y_test

# Modeling and Performance metrics

In [11]:
# Function to train and evaluate models with multiple metrics
def evaluate_models_with_metrics(models, X_train, y_train, X_test, y_test):
    results = []

    for name, model in models:      
        # Train the model
        model.fit(X_train, y_train)
        
        # Predict on test set
        y_test_pred = model.predict(X_test)

        # Store results for the model
        model_results = {
            'Model': name,
            'Test Accuracy': accuracy_score(y_test, y_test_pred),
            'Test Precision': precision_score(y_test, y_test_pred),
            'Test Recall': recall_score(y_test, y_test_pred),
            'Test F1-Score': f1_score(y_test, y_test_pred)
        }
        
        results.append(model_results)

    # Convert results to a pandas DataFrame
    results_df = pd.DataFrame(results)
    
    return results_df

In [12]:
# Create the split
X_train_base, y_train_base, X_test_base, y_test_base = experimental_setup(base_df)

# Evaluate models with multiple metrics and print results
base_results = evaluate_models_with_metrics(ml_models, X_train_base, y_train_base, X_test_base, y_test_base)
display(base_results)

Training Set: X_train shape = (5000, 10000), y_train shape = (5000,)
Test Set: X_test shape = (10000, 10000), y_test shape = (10000,)



,Model,Test Accuracy,Test Precision,Test Recall,Test F1-Score
0,Logistic Regression,0.8502,0.842523,0.864259,0.853252
1,Random Forest,0.8367,0.831323,0.847986,0.839572
2,SVM,0.8346,0.804461,0.887478,0.843933
3,KNN,0.5897,0.570823,0.748561,0.647720
4,Gradient Boosting,0.8026,0.773319,0.860488,0.814578


# Data Summarization

### Non-Qualitative loop

In [13]:
# Prompt
prompt = '''Summarize the following review concisely. Return only the summarized text, without additional commentary or formatting.

Review:
{review}'''

In [14]:
# Initialize a dictionary to store results per model
model_list = {}

# Important! You must only sample from the train set, you mustn't sample from the test set
# This is to prevent data leakage
X_train, _, y_train, _ = train_test_split(base_df['review'], base_df['sentiment'], test_size=0.2, random_state=42)
llm_train_df = pd.DataFrame({'review': X_train, 'sentiment': y_train})

# Iterate through models
for index, model in enumerate(models):
    print("Processing Model: " + model + " (Model " + str(index + 1) + "/" + str(len(models)) + ")")
    model_list[model] = []

    # Summarize n_rows
    for index, row in tqdm(llm_train_df[:n_rows].iterrows(), total=n_rows, desc="Summarizing rows"):
        # Format the prompt baesd on the row
        formatted_prompt = prompt.format(review=row['review'])

        # Summarize
        response = ollama.generate(model=model, prompt=formatted_prompt)['response']

        # Store response and target variable
        model_list[model].append([response, row['sentiment']])

Processing Model: llama3.2:1b (Model 1/8)


Summarizing rows: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [1:17:14<00:00,  1.08it/s]


Processing Model: llama3.2:3b (Model 2/8)


Summarizing rows: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [2:39:18<00:00,  1.91s/it]


Processing Model: gemma3:1b (Model 3/8)


...

Summarizing rows: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [19:39:06<00:00, 14.15s/it]


Processing Model: mistral (Model 7/8)


Summarizing rows: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [10:25:04<00:00,  7.50s/it]


Processing Model: deepseek-llm (Model 8/8)


Summarizing rows: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [12:44:09<00:00,  9.17s/it]


# Comparison

In [15]:
def compare(base_df, llm_df):
    # Set 'Model' as the index in DataFrames
    base_df_results = base_df.set_index('Model')
    llm_df_results = llm_df.set_index('Model')
    
    # Rename evaluation metrics columns
    base_df_results = base_df_results.rename(columns={
        'Test Accuracy': 'Test Accuracy base', 'Test F1-Score': 'Test F1-Score base',
        'Test Precision': 'Test Precision base', 'Test Recall': 'Test Recall base'
    })

    llm_df_results = llm_df_results.rename(columns={
        'Test Accuracy': 'Test Accuracy llm altered', 'Test F1-Score': 'Test F1-Score llm altered',
        'Test Precision': 'Test Precision llm altered', 'Test Recall': 'Test Recall llm altered'
    })
    
    # Merge dfs by index
    raw_final_results = base_df_results.join(llm_df_results)

    # Show raw data results
    print("Raw performance metrics:")
    display(raw_final_results)
    
    # Calculate and display the difference matrix (% difference)
    diff_df = pd.DataFrame()
    diff_df['Test Accuracy % diff'] = round(((raw_final_results['Test Accuracy llm altered'] - raw_final_results['Test Accuracy base']) 
                                                   / raw_final_results['Test Accuracy base']) * 100, 1)
    diff_df['Test F1-Score % diff'] = round(((raw_final_results['Test F1-Score llm altered'] - raw_final_results['Test F1-Score base']) 
                                                 / raw_final_results['Test F1-Score base']) * 100, 1)
    diff_df['Test Precision % diff'] = round(((raw_final_results['Test Precision llm altered'] - raw_final_results['Test Precision base']) 
                                           / raw_final_results['Test Precision base']) * 100, 1)
    diff_df['Test Recall % diff'] = round(((raw_final_results['Test Recall llm altered'] - raw_final_results['Test Recall base']) 
                                           / raw_final_results['Test Recall base']) * 100, 1)
    print("% Difference df (between base and LLM altered df):")
    display(diff_df)
    print()

In [16]:
# Iterate through models
for index, model in enumerate(models):
    print("Processing Model: " + model + " (Model " + str(index + 1) + "/" + str(len(models)) + ")")

    # Get data and make a DataFrame
    summarization_df = pd.DataFrame(model_list[model], columns=['review', 'sentiment'])

    # Split into X and y
    X_train_summarization = summarization_df['review']
    y_train_summarization = summarization_df['sentiment']

    # Get original test set
    _, X_test_base, _, y_test_base = train_test_split(base_df['review'], base_df['sentiment'], test_size=0.2, random_state=42)
    
    # Transform text reviews to Bag of Words representation
    X_train_summarization = vectorizer.fit_transform(X_train_summarization)
    X_test_base = vectorizer.transform(X_test_base)

    # Print the shapes of the sets
    print(f"Training Set: X_train shape = {X_train_summarization.shape}, y_train shape = {y_train_summarization.shape}")
    print(f"Test Set: X_test shape = {X_test_base.shape}, y_test shape = {y_test_base.shape}")
    print()
    
    # For the first n_rows rows
    summarization_tokens = summarization_df['review'][:n_rows].apply(count_tokens_from_string).sum()
    print("Total tokens in base df: ", baseline_1k_rows_tokens) 
    print("Total tokens in summarized df: ", summarization_tokens) 
    print("Reduction: ", round((1 - (summarization_tokens / baseline_1k_rows_tokens)) * 100, 1), "%")
    print()

    # Evaluate models with multiple metrics and display results
    results_summarization = evaluate_models_with_metrics(ml_models, X_train_summarization, y_train_summarization, X_test_base, y_test_base)

    # Compare the results between base and LLM altered df
    compare(base_results, results_summarization)

Processing Model: llama3.2:1b (Model 1/8)
Training Set: X_train shape = (5000, 10000), y_train shape = (5000,)
Test Set: X_test shape = (10000, 10000), y_test shape = (10000,)

Total tokens in base df:  1426591
Total tokens in summarized df:  257535
Reduction:  81.9 %

Raw performance metrics:


,Test Accuracy base,Test Precision base,Test Recall base,Test F1-Score base,Test Accuracy llm altered,Test Precision llm altered,Test Recall llm altered,Test F1-Score llm altered
Model,,,,,,,,
Logistic Regression,0.8502,0.842523,0.864259,0.853252,0.7983,0.756711,0.883906,0.815378
Random Forest,0.8367,0.831323,0.847986,0.839572,0.7935,0.801990,0.783687,0.792733
SVM,0.8346,0.804461,0.887478,0.843933,0.5836,0.548178,0.987894,0.705099
KNN,0.5897,0.570823,0.748561,0.647720,0.6016,0.683478,0.389958,0.496588
Gradient Boosting,0.8026,0.773319,0.860488,0.814578,0.7389,0.712386,0.808097,0.757229


% Difference df (between base and LLM altered df):


,Test Accuracy % diff,Test F1-Score % diff,Test Precision % diff,Test Recall % diff
Model,,,,
Logistic Regression,-6.1,-4.4,-10.2,2.3
Random Forest,-5.2,-5.6,-3.5,-7.6
SVM,-30.1,-16.5,-31.9,11.3
KNN,2.0,-23.3,19.7,-47.9
Gradient Boosting,-7.9,-7.0,-7.9,-6.1



Processing Model: llama3.2:3b (Model 2/8)
Training Set: X_train shape = (5000, 10000), y_train shape = (5000,)
Test Set: X_test shape = (10000, 10000), y_test shape = (10000,)

Total tokens in base df:  1426591
Total tokens in summarized df:  282083
Reduction:  80.2 %

Raw performance metrics:


...

,Test Accuracy % diff,Test F1-Score % diff,Test Precision % diff,Test Recall % diff
Model,,,,
Logistic Regression,-14.7,-9.1,-21.7,8.8
Random Forest,-7.8,-5.6,-12.2,2.1
SVM,-29.1,-15.8,-31.3,11.9
KNN,-0.6,6.8,-3.0,23.1
Gradient Boosting,-17.6,-9.7,-21.6,8.7



Processing Model: deepseek-llm (Model 8/8)
Training Set: X_train shape = (5000, 10000), y_train shape = (5000,)
Test Set: X_test shape = (10000, 10000), y_test shape = (10000,)

Total tokens in base df:  1426591
Total tokens in summarized df:  347941
Reduction:  75.6 %

Raw performance metrics:


,Test Accuracy base,Test Precision base,Test Recall base,Test F1-Score base,Test Accuracy llm altered,Test Precision llm altered,Test Recall llm altered,Test F1-Score llm altered
Model,,,,,,,,
Logistic Regression,0.8502,0.842523,0.864259,0.853252,0.7864,0.737990,0.893233,0.808224
Random Forest,0.8367,0.831323,0.847986,0.839572,0.7890,0.757336,0.855328,0.803355
SVM,0.8346,0.804461,0.887478,0.843933,0.5954,0.555493,0.986307,0.710711
KNN,0.5897,0.570823,0.748561,0.647720,0.6198,0.618600,0.640206,0.629218
Gradient Boosting,0.8026,0.773319,0.860488,0.814578,0.7136,0.660091,0.889859,0.757945


% Difference df (between base and LLM altered df):


,Test Accuracy % diff,Test F1-Score % diff,Test Precision % diff,Test Recall % diff
Model,,,,
Logistic Regression,-7.5,-5.3,-12.4,3.4
Random Forest,-5.7,-4.3,-8.9,0.9
SVM,-28.7,-15.8,-30.9,11.1
KNN,5.1,-2.9,8.4,-14.5
Gradient Boosting,-11.1,-7.0,-14.6,3.4
